In [2]:
import numpy as np
from scipy.fft import fft, ifft
import librosa as rs

def hann_window(length):
    return 0.5 * (1 - np.cos(2 * np.pi * np.arange(length) / (length - 1)))

def asymmetric_windows(K, M, d):
    # Analysis Window  - long window
    analysis_window = np.zeros(K)
    analysis_window[d:K-M] = np.sqrt(hann_window(2 * (K - M - d)))[:K-M-d]
    analysis_window[K-M:] = np.sqrt(hann_window(2 * M))
    
    # Synthesis Window - Short window
    synthesis_window = np.zeros(K)
    synthesis_window[K-2*M:K-M] = hann_window(2 * M)[:M] / analysis_window[K-2*M:K-M]
    synthesis_window[K-M:] = np.sqrt(hann_window(2 * M))
    
    return analysis_window, synthesis_window



def stft(signal, analysis_window, K, M):
    frames = []
    for i in range(0, len(signal) - K + 1, M):
        frame = signal[i:i + K] * analysis_window
        frames.append(fft(frame))
    return np.array(frames)

def istft(frames, synthesis_window, K, M):
    signal = np.zeros((frames.shape[0] - 1) * M + K)
    for i, frame in enumerate(frames):
        time_frame = np.real(ifft(frame)) * synthesis_window
        signal[i * M:i * M + K] += time_frame
    return signal

In [4]:
signla,sr = rs.load("data/female_1.wav")


# Analysis Phase
analysis_window, synthesis_window = asymmetric_windows(K=512, M=128, d=64)
stft_frames = stft(signal, analysis_window, K=512, M=128)


# Synthesis Phase
reconstructed_signal = istft(stft_frames, synthesis_window, K=512, M=128)

ValueError: could not broadcast input array from shape (256,) into shape (128,)